<a href="https://colab.research.google.com/github/BrajanNieto/KNN-HPC/blob/main/Proyecto_HPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Computación Paralela — k-NN distribuido con MPI (mpi4py)**

---

*   Este notebook aplica conceptos de cómputo distribuido con MPI para acelerar el algoritmo **k-Nearest Neighbors (k-NN)**. El flujo implementado: genera un dataset multicategoría, distribuye el conjunto de entrenamiento entre procesos (Scatter), difunde el conjunto de prueba (Bcast), calcula distancias localmente por proceso, reúne resultados parciales (Gather) y aplica voto mayoritario para la predicción final.  
El objetivo es medir tiempos de envío, cómputo y recolección y analizar cómo escala el k-NN usando `mpi4py`. El rank 0 opcionalmente proyecta el dataset con PCA para visualizar etiquetas reales vs predicciones.


**Autores:**  

Nieto Espinoza, Brajan E.  
[brajan.nieto@utec.edu.pe](mailto:brajan.nieto@utec.edu.pe)

Tapia Chasquibol,  Mateo J.  
[mateo,.tapia@utec.edu.pe](mailto:mateo.tapia@utec.edu.pe)

<img src="https://pregrado.utec.edu.pe/sites/default/files/logo-utec-h_0_0.svg" width="190" alt="Logo UTEC" loading="lazy" typeof="foaf:Image">      

---

### 0. Librerías

In [ ]:
!pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.8 MB/s eta 0:00:00


In [ ]:
from mpi4py import MPI
import numpy as np
from collections import Counter
import sys

### Imports Condicionales
Solo el rank 0 importa sklearn (generación de datos y PCA) y la parte de visualización (comentada).


In [ ]:
if MPI.COMM_WORLD.Get_rank() == 0:
    from sklearn.datasets import make_classification
    from sklearn.decomposition import PCA


### Función de distancia
Devuelve un vector con las distancias euclidianas de cada fila de a hacia b.


In [ ]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2, axis=1))

### Configuración MPI



*   comm: comunicador global con todos los procesos.
*   rank: id del proceso actual (0 a size-1).
*   size: número total de procesos lanzados (por ej. -np 4 ⇒ 4).

In [ ]:
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### Parametros



Orden de argumentos:

  

*   argv[1] = train_size (opcional; default 1000)
*   argv[2] = test_size  (opcional; default 200)
*   argv[3] = k          (opcional; default 3)
*   argv[4] = num_features (opcional; default 10)

In [ ]:
train_size   = int(sys.argv[1]) if len(sys.argv) >= 2 else 1000
test_size    = int(sys.argv[2]) if len(sys.argv) >= 3 else 200
k            = int(sys.argv[3]) if len(sys.argv) >= 4 else 3
num_features = int(sys.argv[4]) if len(sys.argv) >= 5 else 10

### DataSet Sintetico



*   Crea un dataset sintético multicategoría (3 clases), solo en rank = 0
*   Se separa en entrenamiento y prueba




In [ ]:
if rank == 0:
    X, y = make_classification(
        n_samples=train_size + test_size,
        n_features=num_features,
        n_informative=6,  # Important: informative features
        n_redundant=0,
        n_classes=3,
        random_state=42,
    )
    X_train = X[:train_size]
    y_train = y[:train_size]
    X_test = X[train_size:]
    y_test = y[train_size:]
else:
    X_train = y_train = X_test = y_test = None

### Difusión broadcast del conjunto de prueba

In [ ]:
X_test = comm.bcast(X_test, root=0)
y_test = comm.bcast(y_test, root=0)

### División scatter del entrenamiento

In [ ]:
local_train_size = train_size // size
local_X = np.empty((local_train_size, num_features), dtype='float64')
local_y = np.empty(local_train_size, dtype='int')

In [ ]:
t_start = MPI.Wtime()
comm.Scatter(X_train, local_X, root=0)
comm.Scatter(y_train, local_y, root=0)
t_dist = MPI.Wtime()

### Cómputo local de vecinos

In [ ]:
local_predictions = []
for x in X_test:
    dists = euclidean_distance(local_X, x)
    k_indices = dists.argsort()[:k]
    k_labels = local_y[k_indices]
    local_predictions.append((dists[k_indices], k_labels))
t_comp = MPI.Wtime()

### Recolección (gather) de candidatos

In [ ]:
all_dists = comm.gather(local_predictions, root=0)
t_gather = MPI.Wtime()

### Fusión global y voto (solo rank 0)

In [ ]:
if rank == 0:
    final_preds = []
    for i in range(test_size):
        all_neighbors = []
        for proc_preds in all_dists:
            all_neighbors.extend(zip(proc_preds[i][0], proc_preds[i][1]))
        all_neighbors.sort(key=lambda x: x[0])
        top_k = [label for _, label in all_neighbors[:k]]
        final_pred = Counter(top_k).most_common(1)[0][0]
        final_preds.append(final_pred)

    final_preds = np.array(final_preds)
    accuracy = np.mean(final_preds == y_test)

    print(f"[Process Count: {size}] Train Size: {train_size} | Test Size: {test_size} | k={k} | d={num_features}")
    print(f"Total Time       : {t_gather - t_start:.4f} sec")
    print(f"  - Distribution : {t_dist - t_start:.4f} sec")
    print(f"  - Computation  : {t_comp - t_dist:.4f} sec")
    print(f"  - Gathering    : {t_gather - t_comp:.4f} sec")
    print(f"Accuracy         : {accuracy:.4f}")
    print(f"Etiquetas reales : {y_test}")
    print(f"Predicciones     : {final_preds}")

    # Visualize results with PCA (no plotting here, solo ejemplo de uso)
    pca = PCA(n_components=2)
    X_test_2D = pca.fit_transform(X_test)